# Introduction

The Ugrainium recipe uses groupings of ingredients to try to cover nutritional requirements with enough overlap that a single ingredient with quality issues does not cause a failure for the whole recipe. A opimizer is used to find the right amount of each ingredient to fulfill the nutritional and practicle requirements for a recipe.

# To Do

* Nutrients without an upper limit should have the upper limit constraint removed
* Add constraints for the NIH essential protein combinations as a limit
* Add a radar graph for vitamins showing the boundry between RDI and UL
* Add a radar graph for vitamins without an upper limit but showing the RDI
* Add a radar graph for essential proteins showing the range between RDI and UL
* Add a radar graph for essential proteins without an upper limit, but showing the RDI as the lower limit
* Add a radar graph pair for non-essential proteins with the above UL and no UL pairing
* Add equality constraints for at least energy, and macro nutrients if possible

In [12]:
# Import all of the helper libraries 

from scipy.optimize import minimize
from scipy.optimize import Bounds
from scipy.optimize import linprog
import pandas as pd
import numpy as np
import os
import json
from math import e

In [13]:
# Setup the notebook context

data_dir = '../data'
pd.set_option('max_columns', 70)


In [108]:
# Download our nutrition profile data from Google Sheets
!(cd ..; make sync_data_from_google)

nutrition_profile_df = pd.read_csv(f'{data_dir}/raw/nutrition_profile.csv', index_col=0, verbose=True).transpose()
ingredients_df = pd.read_csv(f'{data_dir}/raw/ingredients.csv', index_col=0, verbose=True).transpose()

for col in ingredients_df.columns:
    ingredients_df[col] = ingredients_df[col].astype(float)

curl 'https://docs.google.com/spreadsheets/d/1UOx5ZkNMjcwybx9TgFS_EaQnjbokqDx6K9NW0mRDGlg/export?format=csv&id=1UOx5ZkNMjcwybx9TgFS_EaQnjbokqDx6K9NW0mRDGlg&gid=624419712' > data/raw/nutrition_profile.csv
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3719    0  3719    0     0   5940      0 --:--:-- --:--:-- --:--:--  5931
curl 'https://docs.google.com/spreadsheets/d/1UOx5ZkNMjcwybx9TgFS_EaQnjbokqDx6K9NW0mRDGlg/export?format=csv&id=1UOx5ZkNMjcwybx9TgFS_EaQnjbokqDx6K9NW0mRDGlg&gid=1812860789' > data/raw/ingredients.csv
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4877    0  4877    0     0   8693      0 --:--:-- --:--:-- --:--:--  8693
Tokenization took: 0.06 ms
Type conversion took: 1.94 ms
Parser memory cleanup took: 0.01 ms
Tokenization took: 0.02 ms
Typ

In [114]:
nutrition_profile_df

,Calories (kcal),Carbohydrates (g),Protein (g),Total Fat (g),Saturated Fat (g),Monounsaturated Fat (g),Polyunsaturated Fat (g),Omega-3 Fatty Acids (g),Omega-6 Fatty Acids (g),Total Fiber (g),Soluble Fiber (g),Insoluble Fiber (g),Cholesterol (mg),Vitamin A (IU),Vitamin B6 (mg),Vitamin B12 (ug),Vitamin C (mg),Vitamin D (IU),Vitamin E (IU),Vitamin K (ug),Thiamin (mg),Riboflavin (mg),Niacin (mg),Folate (ug),Pantothenic Acid (mg),Biotin (ug),Choline (mg),Calcium (g),Chloride (g),Chromium (ug),Copper (mg),Iodine (ug),Iron (mg),Magnesium (mg),Manganese (mg),Molybdenum (ug),Phosphorus (g),Potassium (g),Selenium (ug),Sodium (g),Sulfur (g),Zinc (mg),Cystine (mg),Histidine (mg),Isoleucine (mg),Leucine (mg),Lysine (mg),Methionine (mg),Phenylalanine (mg),Threonine (mg),Tryptophan (mg),Valine (mg),Tyrosine (mg),Arginine (mg),Alanine (mg),Aspartic acid (mg),Glutamic acid (mg),Glycine (mg),Proline (mg),Serine (mg),Hydroxyproline (mg)
RDI,2000,279,104,57,NaN,NaN,NaN,8.5,12,38,3,30,NaN,3000,1.7,2.4,90,600,20,120,1.2,1.3,16,400,5,30,550,1,2.3,35,0.9,150,8,420,2.3,45,0.7,4.7,55,1.5,0.8,11,816,780,650,910,2176,884,910,455,812,650,875,400,200,500,500,1000,500,500,1
RDI Mask,False,False,False,False,True,True,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,True,False,False,False,False
UL,2000.1,300,300,70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10000,100,NaN,2000,4000,1000,NaN,NaN,NaN,41,1000,NaN,NaN,3500,2.5,NaN,NaN,10,1100,45,NaN,11,2000,4,NaN,400,2.3,2,40,1e+06,NaN,NaN,NaN,300000,80000,NaN,100000,400000,NaN,NaN,2000,NaN,2000,3000,10000,NaN,NaN,NaN
UL Mask,False,False,False,False,True,True,True,True,True,True,True,True,True,False,False,True,False,False,False,True,True,True,False,False,True,True,False,False,True,True,False,False,False,True,False,False,False,True,False,False,False,False,False,True,True,True,False,False,True,False,False,True,True,True,True,True,True,False,True,True,True
target_fac,1,1,1,1,1,1,1,1.7,1.2,1.7,1.2,1.2,1,1.2,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1,1.7,1.7,1,1,1,1,1.7,1,1,1,1.7,1.7,1,1,1.7,1.7,2,2,2,2,2,2,2,2,2,2,2,1,2,1,1,2,2,2,2
target_scale,1,1,1,1,0,0,0,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,0,1,1,1,1
RDI Base,2000,279,104,57,0,0,0,8.5,12,38,3,30,0,3000,1.7,2.4,90,600,20,120,1.2,1.3,16,400,5,30,550,1,2.3,35,0.9,150,8,420,2.3,45,0.7,4.7,55,1.5,0.8,11,816,780,650,910,2176,884,910,455,812,650,875,400,200,500,500,1000,500,500,1
UL Base,2200,300,300,70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10000,100,NaN,2000,4000,1000,NaN,NaN,NaN,41,1000,NaN,NaN,3500,2.5,NaN,NaN,10,1100,45,NaN,11,2000,4,NaN,400,2.3,2,40,1e+06,NaN,NaN,NaN,300000,80000,NaN,100000,400000,NaN,NaN,2000,NaN,2000,3000,10000,NaN,NaN,NaN
UL Try,2200,300,300,70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10000,100,NaN,2000,4000,1000,NaN,NaN,NaN,41,1000,NaN,NaN,3500,2.5,NaN,NaN,10,1100,45,NaN,11,2000,4,NaN,400,2.3,2,40,1e+06,NaN,NaN,NaN,300000,80000,NaN,100000,400000,NaN,NaN,NaN,NaN,NaN,NaN,10000,NaN,NaN,NaN
target_fac_try,1,1,1,1,1,1,1,1.2,1.2,1.2,1.2,1.2,1,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1,1,1.7,1.7,1.7,1,1.7,1.7,1.7,1.7,1,1.7,1.7,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4


In [115]:
# Constrain ingredients before the optimization process. Many of the ingredients are required for non-nutritional requirements 
# or are being limited to enhance flavor

bounds_df = pd.DataFrame(index=ingredients_df.index, data={'lower': 0, 'upper': None})
bounds_df.loc['Guar gum'] = [0.9 * .01, 0.9 * .01]
bounds_df.loc['Xanthan Gum'] = [0.9 * .01, 0.9 * .01]
bounds_df.loc['Alpha-galactosidase enzyme (Beano)'] = [1.0, 1.0]
bounds_df.loc['Multivitamin'] = [1.0, 1.0]
bounds_df.loc['Corn flour, nixtamalized'] = [0, 1.0]
bounds_df.loc['Whey protein'] = [0,0.15]
bounds_df.loc['Ascorbic acid'] = [0.01, 0.01]
bounds = list(zip(bounds_df.lower.values, bounds_df.upper.values))

if In * a = N

where __In__ is the ingredients matrix, __a__ is the amount of each ingredient in the recipe, and __N__ is our nutrient profile, then

inverse(In) * In * a = inverse(In) * N --> a = inverse(In) * N

This solves for __a__ which is the amount needed of each ingredient for our recipe to satisfy our nutrition profile


In [29]:
# Calculate the inverse of the ingredients matrix
ing_inv = pd.DataFrame(np.linalg.pinv(ingredients_df.transpose().values), ingredients_df.index, ingredients_df.columns)


In [27]:
N = nutrition_profile_df.loc['RDI'].values
N.shape

a = np.dot(ing_inv, N)
a

array([ 5.03865457e-01,  3.24274746e-01,  8.90841953e-01, -9.65742783e+01,
        5.34226747e+00,  4.15264758e+00, -1.04682335e-14,  2.68241019e-01,
        8.84004424e-01,  1.16218875e+00, -2.50329415e+00, -4.09177411e-01,
        5.08473250e-04, -4.77326639e-01, -1.51324606e-02, -1.97418513e-01,
        1.38781213e+00,  6.27327885e-01,  1.27652384e-01, -4.07385398e+00])

The solution for __a__ contains negative values for the ingredient recipe, which is not feasible. Let's try another method next.

In [110]:
# Using the scipy linprog optimizer


# Define the cost function. This can be actual cost of ingredients or a measure of the non-objectionability of ingredients to minimize
c = np.ones(ingredients_df.shape[0])

# Define the inequality constraint matrix. This is the ingredients_df data in two parts, the first
# for constraining nutrients to be greater than the RDI, the second to constrain to be less than the UL

# This uses the property: if u < v then -u > -v, to recast less-than into greater-than relationship
A = -ingredients_df.transpose()[nutrition_profile_df.loc['RDI Mask'] == False]
b = -nutrition_profile_df.loc['RDI Base'][nutrition_profile_df.loc['RDI Mask'] == False]

In [111]:
# The upper bound inequality constraint vector. This is the nutrient UL (upper limit)

A = A.append(ingredients_df.transpose()[nutrition_profile_df.loc['UL Mask'] == False]).values
b = b.append(nutrition_profile_df.loc['UL Base'][nutrition_profile_df.loc['UL Mask'] == False]).values

In [105]:
#nutrition_profile_df.loc['UL Try'][57] = 10000.0
#nutrition_profile_df.loc['UL Try'][56] = np.NaN
# Arginine @ 2000 fails
# Aspartic acid @ 2000 fails
# Glutamic acid @ 3000 fails
# Glycine @ 10000 succeeds
#nutrition_profile_df.loc['UL Try'][52:61] 

In [112]:
solution = linprog(c, A_ub=A, b_ub=b, bounds=bounds)
solution.success

True

In [113]:
# Scale the ingredient nutrient amounts for the given quantity of each ingredient given by the optimizer
solution_df = ingredients_df.mul(solution.x, axis=0)
solution_df.insert(0, 'Quantity (g)', solution.x * 100)

# Add a row showing the sum of the scaled amount of each nutrient
total = solution_df.sum()
total.name = 'Total'
solution_df = solution_df.append(total)
solution_df

,Quantity (g),Calories (kcal),Carbohydrates (g),Protein (g),Total Fat (g),Saturated Fat (g),Monounsaturated Fat (g),Polyunsaturated Fat (g),Omega-3 Fatty Acids (g),Omega-6 Fatty Acids (g),Total Fiber (g),Soluble Fiber (g),Insoluble Fiber (g),Cholesterol (mg),Vitamin A (IU),Vitamin B6 (mg),Vitamin B12 (ug),Vitamin C (mg),Vitamin D (IU),Vitamin E (IU),Vitamin K (ug),Thiamin (mg),Riboflavin (mg),Niacin (mg),Folate (ug),Pantothenic Acid (mg),Biotin (ug),Choline (mg),Calcium (g),Chloride (g),Chromium (ug),Copper (mg),Iodine (ug),Iron (mg),Magnesium (mg),Manganese (mg),Molybdenum (ug),Phosphorus (g),Potassium (g),Selenium (ug),Sodium (g),Sulfur (g),Zinc (mg),Cystine (mg),Histidine (mg),Isoleucine (mg),Leucine (mg),Lysine (mg),Methionine (mg),Phenylalanine (mg),Threonine (mg),Tryptophan (mg),Valine (mg),Tyrosine (mg),Arginine (mg),Alanine (mg),Aspartic acid (mg),Glutamic acid (mg),Glycine (mg),Proline (mg),Serine (mg),Hydroxyproline (mg)
Guar gum,0.900000,2.988000,6.957000e-01,0.041400,0.004500,0.000540,0.001980,0.000630,9.000000e-06,0.000603,6.957000e-01,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,2.646000e-03,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.125000e-03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Vitamin K,15.477915,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,77.389575,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000e+00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Oat flour,94.204845,376.819380,6.123315e+01,16.485848,7.065363,1.507278,2.731941,3.108760,1.365970e-01,3.000424,9.420484e+00,3.768194,5.652291,0.0,0.000000,0.117756,0.0,0.000000,0.000000,0.942048,3.014555,0.659434,0.117756,1.413073,30.145550,0.188410,0.0,28.167249,6.123315e-02,0.000000,0.0,0.411675,0.000000,3.768194,135.654977,3.768194,0.0,0.425806,0.349500,32.029647,1.789892e-02,0.000000,3.014555,384.355767,381.529622,653.781624,1209.590209,660.375963,293.919116,843.133362,541.677858,220.439337,882.699397,531.315325,1110.675122,821.466248,1352.781573,3502.536135,796.972988,869.510719,700.884046,0.000000
Choline Bitartrate,0.000004,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000021,0.000000e+00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
"Wheat bran, crude",0.000322,0.000695,2.074340e-04,0.000050,0.000014,0.000002,0.000002,0.000007,5.369939e-07,0.000007,1.376248e-04,0.000000,0.000138,0.0,0.000029,0.000004,0.0,0.000000,0.000000,0.000005,0.000006,0.000002,0.000002,0.000044,0.000254,0.000007,0.0,0.000239,2.347339e-07,0.000000,0.0,0.000003,0.000000,0.000034,0.001965,0.000016,0.0,0.000003,0.000004,0.000250,6.431065e-09,0.000002,0.000023,0.001193,0.001383,0.001563,0.002984,0.001929,0.000752,0.001913,0.001608,0.000907,0.002334,0.001402,0.003495,0.002460,0.003634,0.009241,0.002888,0.002836,0.002199,0.000000
Lecithin granules,2.149903,15.049324,2.149903e-01,0.000000,1.074952,0.322486,0.000000,0.752466,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.0,0.000000,0.000000,0

In [626]:
nutrition_profile_df

,Calories (kcal),Carbohydrates (g),Protein (g),Total Fat (g),Saturated Fat (g),Monounsaturated Fat (g),Polyunsaturated Fat (g),Omega-3 Fatty Acids (g),Omega-6 Fatty Acids (g),Total Fiber (g),Soluble Fiber (g),Insoluble Fiber (g),Cholesterol (mg),Vitamin A (IU),Vitamin B6 (mg),Vitamin B12 (ug),Vitamin C (mg),Vitamin D (IU),Vitamin E (IU),Vitamin K (ug),Thiamin (mg),Riboflavin (mg),Niacin (mg),Folate (ug),Pantothenic Acid (mg),Biotin (ug),Choline (mg),Calcium (g),Chloride (g),Chromium (ug),Copper (mg),Iodine (ug),Iron (mg),Magnesium (mg),Manganese (mg),Molybdenum (ug),Phosphorus (g),Potassium (g),Selenium (ug),Sodium (g),Sulfur (g),Zinc (mg),Cystine (mg),Histidine (mg),Isoleucine (mg),Leucine (mg),Lysine (mg),Methionine (mg),Phenylalanine (mg),Threonine (mg),Tryptophan (mg),Valine (mg),Tyrosine (mg),Arginine (mg),Alanine (mg),Aspartic acid (mg),Glutamic acid (mg),Glycine (mg),Proline (mg),Serine (mg),Hydroxyproline (mg)
RDI,2000.0,279.0,104.0,57.0,0.0,0.0,0.0,14.5,20.4,64.6,5.1,51.0,0.0,5100.0,2.2,4.1,153.0,1020.0,34.0,204.0,2.0,2.2,27.2,680.0,8.5,51.0,935.0,1.7,3.9,35.0,0.9,150.0,8.0,420.0,2.3,45.0,0.7,4.7,55.0,1.5,0.8,11.0,816.0,780.0,650.0,910.0,2176.0,884.0,910.0,455.0,812.0,650.0,875.0,400.0,200.0,500.0,500.0,1000.0,500.0,500.0,1.0
UL,2200.0,300.0,300.0,70.0,NaN,NaN,NaN,100.0,100.0,400.0,300.0,300.0,NaN,10000.0,100.0,240.0,2000.0,4000.0,1000.0,1200.0,50.0,50.0,41.0,1000.0,500.0,500.0,3500.0,2.5,500.0,350.0,10.0,1100.0,45.0,10000.0,11.0,2000.0,4.0,50.0,400.0,2.3,2.0,40.0,1000000.0,1000000.0,1000000.0,1000000.0,300000.0,80000.0,1000000.0,100000.0,400000.0,1000000.0,1000000.0,2000.0,1000000.0,2000.0,3000.0,10000.0,1000000.0,1000000.0,1000000.0
target_fac,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.7,1.7,1.7,1.7,1.7,1.0,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
target_scale,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
RDI Base,2000.0,279.0,104.0,57.0,0.0,0.0,0.0,8.5,12.0,38.0,3.0,30.0,0.0,3000.0,1.3,2.4,90.0,600.0,20.0,120.0,1.2,1.3,16.0,400.0,5.0,30.0,550.0,1.0,2.3,35.0,0.9,150.0,8.0,420.0,2.3,45.0,0.7,4.7,55.0,1.5,0.8,11.0,816.0,780.0,650.0,910.0,2176.0,884.0,910.0,455.0,812.0,650.0,875.0,400.0,200.0,500.0,500.0,1000.0,500.0,500.0,1.0
UL Base,2200.0,300.0,300.0,70.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10000.0,100.0,NaN,2000.0,4000.0,1000.0,NaN,NaN,NaN,41.0,1000.0,NaN,NaN,3500.0,2.5,NaN,NaN,10.0,1100.0,45.0,NaN,11.0,2000.0,4.0,NaN,400.0,2.3,2.0,40.0,1000000.0,NaN,NaN,NaN,300000.0,80000.0,NaN,100000.0,400000.0,NaN,NaN,2000.0,NaN,2000.0,3000.0,10000.0,NaN,NaN,NaN
target_fac_try,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.7,1.7,1.7,1.7,1.7,1.0,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.0,1.0,1.7,1.7,1.7,1.0,1.7,1.7,1.7,1.7,1.0,1.7,1.7,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4


In [463]:
ingredients_df

,Calories (kcal),Carbohydrates (g),Protein (g),Total Fat (g),Saturated Fat (g),Monounsaturated Fat (g),Polyunsaturated Fat (g),Omega-3 Fatty Acids (g),Omega-6 Fatty Acids (g),Total Fiber (g),Soluble Fiber (g),Insoluble Fiber (g),Cholesterol (mg),Vitamin A (IU),Vitamin B6 (mg),Vitamin B12 (ug),Vitamin C (mg),Vitamin D (IU),Vitamin E (IU),Vitamin K (ug),Thiamin (mg),Riboflavin (mg),Niacin (mg),Folate (ug),Pantothenic Acid (mg),Biotin (ug),Choline (mg),Calcium (g),Chloride (g),Chromium (ug),Copper (mg),Iodine (ug),Iron (mg),Magnesium (mg),Manganese (mg),Molybdenum (ug),Phosphorus (g),Potassium (g),Selenium (ug),Sodium (g),Sulfur (g),Zinc (mg),Cystine (mg),Histidine (mg),Isoleucine (mg),Leucine (mg),Lysine (mg),Methionine (mg),Phenylalanine (mg),Threonine (mg),Tryptophan (mg),Valine (mg),Tyrosine (mg),Arginine (mg),Alanine (mg),Aspartic acid (mg),Glutamic acid (mg),Glycine (mg),Proline (mg),Serine (mg),Hydroxyproline (mg)
Guar gum,332.0,77.30,4.60,0.50,0.060,0.220,0.070,0.001,0.067,77.3,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000,0.0,0.000,0.000,0.000,0.0,0.000,0.0,0.0,0.294,0.000,0.0,0.000,0.0,0.00,0.0,0.000,0.0,0.000,0.000,0.0,0.125,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Vitamin K,0.0,0.00,0.00,0.00,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000,500.0,0.000,0.000,0.000,0.0,0.000,0.0,0.0,0.000,0.000,0.0,0.000,0.0,0.00,0.0,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Oat flour,400.0,65.00,17.50,7.50,1.600,2.900,3.300,0.145,3.185,10.0,4.0,6.0,0.0,0.0,0.125,0.0,0.0,0.0,1.000,3.2,0.700,0.125,1.500,32.0,0.200,0.0,29.9,0.065,0.000,0.0,0.437,0.0,4.00,144.0,4.000,0.0,0.452,0.371,34.0,0.019,0.000,3.20,408.0,405.0,694.0,1284.0,701.0,312.0,895.0,575.0,234.0,937.0,564.0,1179.0,872.0,1436.0,3718.0,846.0,923.0,744.0,0.0
Choline Bitartrate,0.0,0.00,0.00,0.00,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000,0.0,0.000,0.000,0.000,0.0,0.000,0.0,550.0,0.000,0.000,0.0,0.000,0.0,0.00,0.0,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Wheat bran, crude",216.0,64.51,15.55,4.25,0.630,0.637,2.212,0.167,2.039,42.8,0.0,42.8,0.0,9.0,1.303,0.0,0.0,0.0,1.490,1.9,0.523,0.577,13.578,79.0,2.181,0.0,74.4,0.073,0.000,0.0,0.998,0.0,10.57,611.0,5.000,0.0,1.013,1.182,77.6,0.002,0.605,7.27,371.0,430.0,486.0,928.0,600.0,234.0,595.0,500.0,282.0,726.0,436.0,1087.0,765.0,1130.0,2874.0,898.0,882.0,684.0,0.0
Lecithin granules,700.0,10.00,0.00,50.00,15.000,0.000,35.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000,0.0,0.000,0.000,0.000,0.0,0.000,0.0,13000.0,0.200,0.000,0.0,0.000,0.0,3.60,0.0,0.000,0.0,3.000,1.600,0.0,0.000,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Alpha-galactosidase enzyme (Beano),0.0,0.00,0.00,0.00,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000,0.0,0.000,0.000,0.000,0.0,0.000,0.0,0.0,0.000,0.000,0.0,0.000,0.0,0.00,0.0,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Canola oil,884.0,0.00,0.00,100.00,7.365,63.276,28.142,9.137,19.005,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,17.460,71.3,0.000,0.000,0.000,0.0,0.000,0.0,0.2,0.000,0.000,0.0,0.000,0.0,0.00,0.0,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Pea Protein Powder,375.0,1.00,82.10,7.14,3.570,0.000,0.000,0.000,0.000,0.5,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000,0.0,0.000,0.000,0.000,0.0,0.000,0.0,95.0,0.143,0.000,0.0,0.000,0.0,5.79,0.0,0.000,0.0,0.000,0.000,0.0,1.200,0.000,0.00,870.0,2510.0,4760.0,8410.0,7360.0,980.0,5520.0,4050.0,1030.0,5030.0,2810.0,6540.0,3210.0,9110.0,13530.0,3080.0,3380.0,4170.0,0.0
Multivitamin,0.0,0.00,0.00,0.00,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,3500.0,2.000,6.0,90.0,400.0,30.000,25.0,1.500,1.700,20.000,500.0,10.00

In [33]:
(2206 - 1762) / 67 * 60

397.6119402985074

In [34]:
700 * 4 / 398

7.035175879396985